In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
import pickle

In [ ]:
experiment_name = 'Random Comma 1'
statistics_file = 'D:/experiments/' + experiment_name + '/statistics.pkl'

with open(statistics_file, 'rb') as f:
	stat = pickle.load(f)

def hms(seconds):
	return time.strftime("%H:%M:%S", time.gmtime(seconds))

print(f"{experiment_name}: {stat.run_generation} generations")
print(f"{stat.evaluations_total} evaluations ({stat.evaluations_k_folds} for k-folds), {stat.evaluations_cache_hits} cache hits, {stat.evaluations_invalid} invalid")
print(f"runtime {hms(stat.run_time)}, evaluation time {hms(stat.eval_time)} (this run {hms(stat.eval_time_this_run)}), k-folds: {hms(stat.eval_time_k_folds)} (this run {hms(stat.eval_time_k_folds_this_run)})")

In [ ]:
def plot_accuracy(ax, xscale, ind_stat, color, label):
	if len(ind_stat.k_fold_accuracy):
		ax.plot(xscale, ind_stat.k_fold_accuracy, '*-', color=color, alpha=0.7, label="K-fold " + label)
		ax.errorbar(xscale, ind_stat.k_fold_accuracy, yerr = ind_stat.k_fold_accuracy_std, color=color, alpha=0.7, zorder=10)
	else:
		ax.plot(ind_stat.accuracy, 'o-', color = color, alpha=0.5, label=label)

def plot_fitness(ax, xscale, ind_stat, color, label):
	if len(ind_stat.k_fold_accuracy):
		ax.plot(xscale, ind_stat.k_fold_fitness, '*-', color=color, alpha=0.7, label="K-fold " + label)
		ax.errorbar(xscale, ind_stat.k_fold_fitness, yerr = ind_stat.k_fold_fitness_std, color=color, alpha=0.7, zorder=10)
	else:
		ax.plot(ind_stat.fitness, 'o-', color = color, alpha=0.5, label=label)

# show label for 'Population accuracy' only once
def reduced_legend(ax, population_size, additional_entries=1):
	handles, labels = ax.get_legend_handles_labels()
	display = [0] + [i for i in range(population_size, population_size + additional_entries)]
	ax.legend([handle for i,handle in enumerate(handles) if i in display],
				[label for i,label in enumerate(labels) if i in display], loc = 'best', fontsize=15)
	ax2.grid(True)

generation_accuracy = np.array(stat.generation_accuracy)
generation_parameters = np.array(stat.generation_parameters)
generation_fitness = np.array(stat.generation_fitness)
(ngenerations, population_size) = generation_accuracy.shape
xscale = np.arange(0, stat.run_generation)

fig, ((ax, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(20,14))

ax.set_title('Accuracy', fontsize=25)
for i in range(population_size):
	ax.plot(generation_accuracy[:, i], 'o', color ='#C0C0C0', label = 'population accuracy', zorder=-32)
plot_accuracy(ax, xscale, stat.best, 'blue', 'best')
plot_accuracy(ax, xscale, stat.best_in_gen, 'magenta', 'best in generation')
ax.set_ylim(0.96, 1.0)
ax.set_xlim(0, ngenerations)
reduced_legend(ax, population_size, 2)

ax2.set_title("Number of Parameters", fontsize=25)
for i in range(population_size):
	ax2.plot(generation_parameters[:, i], '*', color ='#C0C0C0', label = 'population', zorder=-32)
ax2.plot(stat.best.parameters, color='blue', label="best")
ax2.plot(stat.best_in_gen.parameters, color='magenta', alpha=0.7, label="best in generation")
ax2.set_xlim(0, ngenerations)
ax2.set_ylim(0, 100000)
reduced_legend(ax2, population_size, 2)

ax3.set_title('Fitness', fontsize=25)
for i in range(population_size):
	ax3.plot(generation_fitness[:, i], '*', color ='#C0C0C0', label = 'Population fitness', zorder=-32)
plot_fitness(ax3, xscale, stat.best, 'blue', 'best')
plot_fitness(ax3, xscale, stat.best_in_gen, 'magenta', 'best in generation')
ax3.set_ylim(-10, 3)
ax3.set_xlim(0, ngenerations)
reduced_legend(ax3, population_size, 2)

ax4.set_title('Different Accuracy Measures', fontsize=25)
if len(stat.best.k_fold_accuracy):
	ax4.plot(xscale, stat.best.k_fold_accuracy, 'b*-', label="avg K-fold accuracy")
	ax4.errorbar(xscale, stat.best.k_fold_accuracy, yerr = stat.best.k_fold_accuracy_std, ecolor='b', zorder=10)
	ax4.plot(stat.k_fold_final_accuracy, label="avg K-fold final accuracy")
ax4.plot(stat.best.accuracy, label="test accuracy")
ax4.plot(stat.best.train_accuracy, label="training accuracy")
ax4.plot(stat.best.val_accuracy, label="validation accuracy")
ax4.plot(stat.best.final_test_accuracy, label="final test accuracy")
ax4.set_ylim(0.96, 1.0)
ax4.set_xlim(0, ngenerations)
ax4.legend(fontsize=15)
ax4.grid(True)